In [30]:
import numpy as np 
import pandas as pd


State="NC"
GeneratorDataPath="february_generator2023.xlsx"


# Code to Convert Excel Table to Sql data

## Generation 


In [ ]:
##Generation data from: Preliminary Monthly Electric Generator Inventory (based on Form EIA-860M as a supplement to Form EIA-860)

#Read File
df = pd.read_excel(GeneratorDataPath, sheet_name ='Operating',skiprows=2)
InState=df["Plant State"]==State #Filter for NC
df=df[InState]

In [161]:
df_AggregateGen.to_excel("tmp.xlsx")


In [160]:
#   Some plants may have multiple generators commissioned at different times
#   we are aggregating the capacity of these generators per year

#   We can have multiple vintages (years) of the same plant. As generators may be comissioned at different times

df_AggregateGen = pd.DataFrame(columns=["PlantID", "SourceCode", "MoverCode", "OperatingYear","NameplateCapacity(MW)","NetSummerCapacity(MW)","NetWinterCapacity(MW)",
 "NameplateEnergyCapacity(MWh)", "Latitude","Longitude","Technology"])

for ID in df["Plant ID"].unique().astype(int):
    #Same Plant data
    SameIds   = df["Plant ID"]==ID
    df_tmp1    = df[SameIds]

    SourceCode=df_tmp1["Energy Source Code"]
    #Same Energy Source Code data
    for UniqueSC in SourceCode.unique():
        SameIds    = df_tmp1["Energy Source Code"]==UniqueSC
        df_tmp2    = df_tmp1[SameIds]

        MoverCode=df_tmp2["Prime Mover Code"]
        #Same Mover Code data
        for UniqueMC in MoverCode.unique():
            SameIds    = df_tmp2["Prime Mover Code"]==UniqueMC
            df_tmp3    = df_tmp2[SameIds]

            OperatingYear=df_tmp3["Operating Year"]
            OperatingYear.sort_values()
            #Same Year Code data
            for UniqueY in OperatingYear.unique():
                SameIds    = df_tmp3["Operating Year"]==UniqueY
                df_tmp4    = df_tmp3[SameIds]

                NameplateEnergyCapacity=np.sum(df_tmp4["Nameplate Energy Capacity (MWh)"])
                NameplateCapacity=np.sum(df_tmp4["Nameplate Capacity (MW)"])
                NetSummerCapacity=np.sum(df_tmp4["Net Summer Capacity (MW)"])
                NetWinterCapacity=np.sum(df_tmp4["Net Winter Capacity (MW)"])

                Latitude  = df_tmp4["Latitude"].iloc[0]
                Longitude = df_tmp4["Longitude"].iloc[0]
                Technology= df_tmp4["Technology"].iloc[0]

                NewPlantID = UniqueSC + "_" + UniqueMC + "_" + str(ID) 

                Data=[[NewPlantID, UniqueSC, UniqueMC, UniqueY, NameplateCapacity, NetSummerCapacity, NetWinterCapacity,NameplateEnergyCapacity,
                Latitude,Longitude,Technology]]

                df_AggregateGen=pd.concat([df_AggregateGen,pd.DataFrame(Data,columns=df_AggregateGen.columns)],ignore_index=True)